In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import pathlib
import numpy as np
from tensorflow.keras import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Input, Add, ReLU, AveragePooling2D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image

path = '/content/gdrive/My Drive/caltech101_pickle'
image_size = (164, 164)
input_shape = (164, 164, 3)
num_classes = 102

In [3]:
import gzip
import pickle

os.chdir(path)

with gzip.open('caltech101_x_trainData.pickle', 'rb') as f:
    x_train = pickle.load(f)
    
with gzip.open('caltech101_x_testData.pickle', 'rb') as f:
    x_test = pickle.load(f)
    
with gzip.open('caltech101_y_trainData.pickle', 'rb') as f:
    y_train = pickle.load(f)
    
with gzip.open('caltech101_y_testData.pickle', 'rb') as f:
    y_test = pickle.load(f)

In [8]:
base_model = ResNet50(include_top = False, weights = 'imagenet')

x = base_model.output

shortcut = x
shortcut = Conv2D(filters = 2048, kernel_size = 1, strides = 1)(shortcut)
shortcut = BatchNormalization(axis = 3, epsilon=1.001e-5)(shortcut)

x = Conv2D(filters = 512, kernel_size = 1, strides = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 1024, kernel_size = 3, padding='SAME')(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5,)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 2048, kernel_size = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)

x = Add()([shortcut, x])
x = Activation('relu')(x)

shortcut = x
shortcut = Conv2D(filters = 2048, kernel_size = 1, strides = 1)(shortcut)
shortcut = BatchNormalization(axis = 3, epsilon=1.001e-5)(shortcut)

x = Conv2D(filters = 512, kernel_size = 1, strides = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 1024, kernel_size = 3, padding='SAME')(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5,)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 2048, kernel_size = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)

x = Add()([shortcut, x])
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)

prediction = Dense(num_classes, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = prediction)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [9]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

In [10]:
model.fit(x_train, y_train, validation_split = 0.1, batch_size = 128, epochs = 20)

Epoch 1/20
52/52 [==============================] - 129s 2s/step - loss: 1.8872 - accuracy: 0.5762 - val_loss: 27.2522 - val_accuracy: 0.0232
Epoch 2/20
52/52 [==============================] - 109s 2s/step - loss: 0.6904 - accuracy: 0.8112 - val_loss: 4.2931 - val_accuracy: 0.2363
Epoch 3/20
52/52 [==============================] - 109s 2s/step - loss: 0.3522 - accuracy: 0.9019 - val_loss: 2.6202 - val_accuracy: 0.5096
Epoch 4/20
52/52 [==============================] - 109s 2s/step - loss: 0.2068 - accuracy: 0.9386 - val_loss: 1.7078 - val_accuracy: 0.6366
Epoch 5/20
52/52 [==============================] - 109s 2s/step - loss: 0.1728 - accuracy: 0.9500 - val_loss: 1.4718 - val_accuracy: 0.6831
Epoch 6/20
52/52 [==============================] - 109s 2s/step - loss: 0.1076 - accuracy: 0.9667 - val_loss: 1.8444 - val_accuracy: 0.6571
Epoch 7/20
52/52 [==============================] - 109s 2s/step - loss: 0.0858 - accuracy: 0.9781 - val_loss: 1.1404 - val_accuracy: 0.7459
Epoch 8/20
5

In [11]:
model.evaluate(x_test, y_test)

58/58 [==============================] - 11s 190ms/step - loss: 0.6669 - accuracy: 0.8464


[0.666911244392395, 0.8463641405105591]